In [ ]:
# Предполагаю, что data_df - датасет точек
# wine_df - датасет виноделен

In [105]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import jaccard_score
from functools import cmp_to_key

In [139]:
data_df = pd.read_csv("lands_without_water.csv").dropna()
wine_df = pd.read_csv("data.csv").dropna()

In [142]:
len(data_df)

160801

In [143]:
wine_df.drop(['admin'], axis=1)

,x,y,landcover,relief_aspect,relief_height,relief_slope,relief_soiltexture,sunny_days,vineyard,water_seasonality
0,197,2445,7,360.000000,17,1.432096,3,111,True,0
1,197,2446,7,18.434948,17,0.905850,3,111,True,0
2,198,2442,5,18.434948,21,0.905850,3,111,True,0
3,198,2443,5,11.309933,20,1.460445,3,111,True,0
4,198,2444,5,11.309933,20,1.460445,3,111,True,0
...,...,...,...,...,...,...,...,...,...,...
35581,2938,1363,5,186.009000,58,10.816006,6,128,True,0
35582,2938,1364,7,169.315100,58,15.092257,6,128,True,0
35583,2939,1362,5,217.954220,35,17.591572,0,128,True,12
35584,2939,1363,5,180.000000,46,10.481218,6,128,True,0


In [144]:
wine_df.head()

,x,y,admin,landcover,relief_aspect,relief_height,relief_slope,relief_soiltexture,sunny_days,vineyard,water_seasonality
0,197,2445,False,7,360.000000,17,1.432096,3,111,True,0
1,197,2446,False,7,18.434948,17,0.905850,3,111,True,0
2,198,2442,False,5,18.434948,21,0.905850,3,111,True,0
3,198,2443,False,5,11.309933,20,1.460445,3,111,True,0
4,198,2444,False,5,11.309933,20,1.460445,3,111,True,0


In [145]:
wineyard = None

In [146]:
def yard_metric(a, b): # старый вариант
  categoric = ['landcover', 'relief_soiltexture', 'water_seasonality'] # список категориальных фич
  numeric = ['relief_aspect', 'relief_height', 'relief_slope', 'sunny_days'] # список некатегориальных фич
  cat = 1
  num = 1 # коэффициенты
  ans = num * euclidean_distances(a[numeric], b[numeric]) + cat * jaccard_score(a[categoric], b[categoric])
  return ans

In [147]:
def yard_metric(a, b):
  categoric = ['landcover', 'relief_soiltexture', 'water_seasonality'] # список категориальных фич
  numeric = ['relief_aspect', 'relief_height', 'relief_slope', 'sunny_days'] # список некатегориальных фич
  ans = int(np.linalg.norm(a[numeric]-b[numeric]) + np.linalg.norm(a[categoric]-b[categoric]))
  return ans

In [148]:
def cmp_wineyards(a, b):
  global wineyard
  if (yard_metric(wineyard, data_df.iloc[a]) < yard_metric(wineyard, data_df.iloc[b])):
    return -1
  elif (yard_metric(wineyard, data_df.iloc[a]) > yard_metric(wineyard, data_df.iloc[b])):
    return 1
  else:
    return 0

In [151]:
def find_k_nearest(k, x_coord, y_coord): # k - количество в списке, x, y - координаты виноградни для сравнения
  global wineyard
  needed_str = wine_df.where(wine_df.x == x_coord).where(wine_df.y == y_coord)
  wineyard = needed_str.iloc[0] # виноградник, от которого ищем
  indices = []  
  for i in range(len(data_df)):
    if (i % 100 == 0):
      print(i)
    indices.append(i)
    if (len(indices) > 2000):
      sorted(indices, key = cmp_to_key(cmp_wineyards))
      indices = indices[:k]
  return indices # возвращает список номеров строк, которые подходят

In [152]:
print(find_k_nearest(10, 197, 2445))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


KeyboardInterrupt: ignored